In [1]:
!nvidia-smi

Thu Aug  5 07:35:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#config consts
DATASET =     {'train':'train.tar.gz',
               'test':'test.tar.gz',
               'train_mask':'train_mask.tar.gz'}

METADATA=     {'image_level':'train_image_level.csv',
               'study_level':'train_study_level.csv',
               'df_meta':'df_meta.csv',
               'df_fold_rand830':'df_fold_rand830.csv',
               'train_dup':'duplicate.txt',
               'sample_sub':'sample_submission.csv',
               'df_annotate':'df_annotate.csv',
               'df_meta_hw':'df_meta_hw.csv',
               'yolov5m':'yolov5m.pt',
               'yolov5x':'yolov5x.pt',
               'submit_0598': 'submit_0598.csv'}

CFGMODEL_DIR_DICT = {'B3_512':'effb3-full-512-mask-v8/',
                     'B4_512':'effb4-full-512-mask/',
                     'B5_600':'effb5-600-mask/',
                     'B5_640':'effb5-640-mask/',
                     'D201_640':'d201-640-mask/',
                     'YOLO5_640':'custom-yolo5-640-v4/'}

INPUT_DIR ='/content/drive/My\ Drive/kaggle/covid19-det/input/'

OUTPUT_DIR = {'BSL':'/content/drive/My\ Drive/kaggle/covid19-det/output/',
              'NORM':'/content/drive/My Drive/kaggle/covid19-det/output/'}

IMPORT_DIR = '/content/drive/My Drive/kaggle/covid19-det/nbs/py/'

HENGCK_IM_DIR=IMPORT_DIR+'hengck_code/dummy_01q/'
HENGCK_IM_DIR_YOLO=IMPORT_DIR+'hengck_code/dummy_01a/'

WORK_DIR='/content/'

DATASET_DIR_DICT = {'256': INPUT_DIR+'256_jpg/',
                    '512': INPUT_DIR+'512_jpg/',
                    '600': INPUT_DIR+'600_jpg/',
                    '640': INPUT_DIR+'640_jpg/'}

EXPERIMENT='SZ640_YOLO_TEST3'
EXPERIMENT_DIR = OUTPUT_DIR['BSL'] + EXPERIMENT+'/'
CFGMODEL_DIR = CFGMODEL_DIR_DICT['YOLO5_640']
MASK_SIZE=(40,40)
DATASET_DIR = DATASET_DIR_DICT['640']
METADATA_DIR = INPUT_DIR+'metadata/'
FOLDS_SET=[0,1,2,3,4]
ENSEMBLE_CSV='/ensyolo_t3.csv'
INITIAL_CHECKPOINTS=['best_model.pth','best_model.pth','best_model.pth','best_model.pth','best_model.pth']

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
if 0:
  !cp -r {OUTPUT_DIR['BSL']+'mpack'} '/content'
  !tar zcvf mpack.tar.gz '/content/mpack/'
  !cp /content/mpack.tar.gz {OUTPUT_DIR['BSL']}

In [5]:
def copy_dataset(ds_dict, ds_dir, work_dir):
  for record in ds_dict:
    print('copy', ds_dir+ds_dict[record], ' to', work_dir)
    !cp {ds_dir+ds_dict[record]} {work_dir}
    print('mkdir',work_dir+record)
    !mkdir {work_dir+record}
    print ('tar -xzf',work_dir+ds_dict[record],'-C',work_dir+record)
    !tar -xzf  {work_dir+ds_dict[record]} -C {work_dir+record}
    print ('rm ',work_dir+ds_dict[record])
    !rm {work_dir+ds_dict[record]}
def copy_metadata(md_dict,md_dir,work_dir):
  for record in md_dict:
    print('copy ', md_dir+md_dict[record],' to ',work_dir)
    !cp {md_dir+md_dict[record]} {work_dir}

copy_dataset(DATASET,DATASET_DIR, WORK_DIR)
copy_metadata(METADATA,METADATA_DIR,WORK_DIR)
!ls /content/

copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/train.tar.gz  to /content/
mkdir /content/train
tar -xzf /content/train.tar.gz -C /content/train
rm  /content/train.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/test.tar.gz  to /content/
mkdir /content/test
tar -xzf /content/test.tar.gz -C /content/test
rm  /content/test.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/train_mask.tar.gz  to /content/
mkdir /content/train_mask
tar -xzf /content/train_mask.tar.gz -C /content/train_mask
rm  /content/train_mask.tar.gz
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_image_level.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_study_level.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/df_meta.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/df_fold_rand830.csv  to  /content/
copy  /content/drive/My\ D

In [6]:
import sys
sys.path.append(HENGCK_IM_DIR_YOLO)
sys.path.append(HENGCK_IM_DIR_YOLO+CFGMODEL_DIR)

!pip install pydicom
!pip install ensemble-boxes
#!pip install madgrad
#!pip install timm

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from siim import *


from model import *
from dataset import *
from common import *
from ensemble_boxes import *
from tqdm import tqdm

     |████████████████████████████████| 1.9 MB 10.8 MB/s 
matplotlib.get_backend :  module://ipykernel.pylab.backend_inline


/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/My Drive/kaggle/covid19-det/nbs/py/hengck_code/dummy_01a/siim/map_boxes/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [7]:
def make_df_image(df_valid, detection):
    df_image = pd.DataFrame()
    df_image.loc[:,'id'] = df_valid.image + '_image'
    #df_image.loc[:, 'PredictionString']=''

    predict_string = []
    for i,det in enumerate(detection):
        d = df_valid.iloc[i]

        s = ''
        for x0, y0, x1, y1, c in det:
            x0 = int(x0/image_size*d.width )
            y0 = int(y0/image_size*d.height)
            x1 = int(x1/image_size*d.width )
            y1 = int(y1/image_size*d.height)

            if x0<0: x0=0
            if y0<0: y0=0
            if x1<0: x1=0
            if y1<0: y1=0

            if x0>d.width: x0=d.width
            if y0>d.height: y0=d.height
            if x1>d.width: x1=d.width
            if y1>d.height: y1=d.height

            if (x1-x0)*(y1-y0) == 0.0: continue

            s += ' opacity %0.5f %4d %4d %4d %4d'%(c,x0,y0,x1,y1)

        predict_string.append(s)

    df_image.loc[:, 'PredictionString'] = predict_string
    #df_image = df_image[['id','PredictionString']]
    return df_image


#---------------------------------------------------------


def do_predict(net, valid_loader, tta=[]): #flip

    valid_detection = []
    valid_num = 0

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        batch_size = len(batch['index'])
        image  = batch['image'].cuda()

        onehot = batch['onehot']
        label  = onehot.argmax(-1)
        mask   = batch['mask']

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            predict = net(image)
            predict = infer_prediction(predict)
            predict_flat = pyramid_to_flat(predict)

            detection = do_non_max_suppression(
                    predict_flat,
                    nms_objectness_threshold=0.0001,
                    nms_iou_threshold=0.5,
                    nms_pre_max_num=500,
                    nms_post_max_num=25,
            )

            #debug ------------------------------------------------------------------
            if 0:
                image = image.permute(0, 2, 3, 1).contiguous()
                image = image.data.cpu().numpy()
                image = (image * 255).astype(np.uint8)

                mask = mask.permute(0, 2, 3, 1).repeat(1, 1, 1, 3)
                mask = mask.data.cpu().numpy()
                mask = (mask * 255).astype(np.uint8)

                for b in range(batch_size):
                    image_show('image', image[b], resize=1)
                    image_show('mask', mask[b], resize=1)

                    overlay = mask[b].copy()
                    d = detection[b]
                    for x0,y0,x1,y1,s in d:
                        x0 = int(x0)
                        y0 = int(y0)
                        x1 = int(x1)
                        y1 = int(y1)
                        if s>0.5:
                            cv2.rectangle(overlay, (x0, y0), (x1, y1), (0, 0, 255), 3)
                            draw_shadow_text(overlay, '%0.4f'%s, (x0,y0+15), 0.8, (255,255,255),1)
                        else:
                            cv2.rectangle(overlay, (x0, y0), (x1, y1), (0, 255, 0), 1)

                    image_show('overlay', overlay, resize=1)
                    cv2.waitKey(0)


        valid_num += batch_size
        valid_detection.extend(detection)
        print('\r %8d / %d  %s' % (valid_num, len(valid_loader.dataset), time_to_str(timer() - start_timer, 'sec')),
              end='', flush=True)

    assert(valid_num == len(valid_loader.dataset))
    print('')

    detection = valid_detection
    return detection

def run_submit():

    for fold in FOLDS_SET:
        out_dir = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d'%fold
        initial_checkpoint =out_dir + '/checkpoint/'+INITIAL_CHECKPOINTS[fold] # None #
            

        if 1:

            ## setup  ----------------------------------------
            #mode = 'local'
            mode = 'remote'

            submit_dir = out_dir + '/valid/%s'%(mode)
            os.makedirs(submit_dir, exist_ok=True)

            log = Logger()
            log.open(out_dir + '/log.submit.txt', mode='a')
            log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
            log.write('\t%s\n' % COMMON_STRING)
            log.write('\n')

            #
            ## dataset ------------------------------------

            if 'remote' in mode: #1263
                df_annotate, df_valid = make_fold('test')

            if 'local' in mode: #1276 #1256
                df_annotate, df_train, df_valid = make_fold('train-%d' % fold)
                #df_valid = df_train

            valid_dataset = SiimDataset(df_annotate, df_valid)
            valid_loader  = DataLoader(
                valid_dataset,
                sampler = SequentialSampler(valid_dataset),
                batch_size  = 4,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = null_collate,
            )
            log.write('mode : %s\n'%(mode))
            log.write('valid_dataset : \n%s\n'%(valid_dataset))

            ## net ----------------------------------------
            if 1:
                net = Net().cuda()
                net.load_state_dict(torch.load(initial_checkpoint)['state_dict'], strict=True)

                #---
                start_timer = timer()
                detection = do_predict(net, valid_loader)
                log.write('time %s \n' % time_to_str(timer() - start_timer, 'min'))
                log.write('detection %d \n' % len(detection))

                write_pickle_to_file(submit_dir + '/detection.pickle',detection)
                #df_valid['study'].to_csv(submit_dir + '/study.csv', index=False)
                #df_valid.to_csv(submit_dir + '/df_valid.csv', index=False)

                #write_pickle_to_file(submit_dir + '/study.pickle', df_valid.study.values)
                #exit(0)
            else:
                detection = read_pickle_from_file(submit_dir + '/detection.pickle')
                pass


            #----
            df_image  = make_df_image(df_valid, detection)
            df_submit = df_image
            df_submit.to_csv(submit_dir + '/submit.csv', index=False)

            log.write('submit_dir : %s\n' % (submit_dir))
            log.write('initial_checkpoint : %s\n' % (initial_checkpoint))
            log.write('df_submit : %s\n' % str(df_submit.shape))
            log.write('%s\n' % str(df_submit))
            log.write('\n')

            if 'local' in mode:
                #exit(0)

                #['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax']
                df_predict = {
                    'ImageID':[],
                    'LabelName':[],
                    'Conf':[],
                    'XMin':[],
                    'XMax':[],
                    'YMin':[],
                    'YMax':[],
                }
                for i,det in enumerate(detection):
                    d = df_valid.iloc[i]

                    for x0, y0, x1, y1, c in det:
                        x0 = int(x0 / image_size * d.width)
                        y0 = int(y0 / image_size * d.height)
                        x1 = int(x1 / image_size * d.width)
                        y1 = int(y1 / image_size * d.height)



                        df_predict['ImageID'].append(d.image)
                        df_predict['LabelName'].append(0)
                        df_predict['Conf'].append(c)
                        df_predict['XMin'].append(x0)
                        df_predict['XMax'].append(x1)
                        df_predict['YMin'].append(y0)
                        df_predict['YMax'].append(y1)

                df_predict = pd.DataFrame(df_predict)
                log.write('df_predict.shape : %s\n' % str(df_predict.shape))


                #-------------------------------------------------------------------------

                #['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']
                df_truth = {
                    'ImageID':[],
                    'LabelName':[],
                    'XMin':[],
                    'XMax':[],
                    'YMin':[],
                    'YMax':[],
                }

                gb = df_annotate.groupby('image_id')
                for i,d in df_valid.iterrows():
                    g = gb.get_group(d.image)

                    for j,f in g.iterrows():
                        if f.class_id==0: continue
                        x0 = f.x
                        y0 = f.y
                        x1 = f.x+f.w
                        y1 = f.y+f.h
                        df_truth['ImageID'].append(d.image)
                        df_truth['LabelName'].append(0)
                        df_truth['XMin'].append(x0)
                        df_truth['XMax'].append(x1)
                        df_truth['YMin'].append(y0)
                        df_truth['YMax'].append(y1)

                df_truth = pd.DataFrame(df_truth)
                log.write('df_truth.shape : %s\n' % str(df_truth.shape))

                map, _ = mean_average_precision_for_boxes(
                    df_truth, df_predict, iou_threshold=0.5, exclude_not_in_annotations=False, verbose=True)

                log.write('map(opacity) : %f\n' % map)
                log.write('map*0.16     : %f\n' % (map/6))
                log.write('\n\n')
        #exit(0)



In [8]:
def run_wbf_submit(nms_objectness_threshold=0.001,nms_pre_max_num=500,iou_thr=0.55,skip_box_thr=0.001):
  for fold in FOLDS_SET:
    out_dir = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d'%fold
    initial_checkpoint =out_dir + '/checkpoint/'+INITIAL_CHECKPOINTS[fold]
    submit_dir = out_dir + '/valid/remote'
    df_annotate, df_valid = make_fold('test')
    valid_dataset = SiimDataset(df_annotate, df_valid)
    valid_loader  = DataLoader(
                valid_dataset,
                sampler = SequentialSampler(valid_dataset),
                batch_size  = 4,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = null_collate,
            )
    net = Net().cuda()
    net.load_state_dict(torch.load(initial_checkpoint)['state_dict'], strict=True)
    df_image = pd.DataFrame()
    df_image.loc[:,'id'] = df_valid.image + '_image'
    img_i=0
    prediction_string=[]
    for t, batch in tqdm(enumerate(valid_loader)):
      image  = batch['image'].cuda()
      net.eval()
      with torch.no_grad():
        predict = net(image)
        predict = infer_prediction(predict)
        predict_flat = pyramid_to_flat(predict)
        batch_size = len(batch['index'])
        for b in range(batch_size):
          d = df_valid.iloc[img_i]
          #print(d.image,batch['d'][b]['image'],'\n')
          img_i=img_i+1
          p=predict_flat[b]
          i = p[..., 4] > nms_objectness_threshold
          p=p[i]
          num=len(p)
          if num==0:
            prediction_string.append('none 1 0 0 1 1')
            continue
          boxes = xywh2xyxy(p[:, :4])
          scores = p[:,4]
          if num > nms_pre_max_num:  # excess boxes
            i = scores.argsort(descending=True)[:nms_pre_max_num]  # sort by confidence
            boxes = boxes[i]
            scores = scores[i]
          labels = torch.zeros_like(scores)
          wbf_boxes, wbf_scores, wbf_labels = weighted_boxes_fusion([(boxes/image_size).tolist()], [scores.tolist()], [labels.tolist()],
                                                                    weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
          det = np.concatenate([wbf_boxes,wbf_scores[:,None]],-1)
          s=''
          for x0, y0, x1, y1, c in det:
            x0 = int(round(x0*d.width) )
            y0 = int(round(y0*d.height))
            x1 = int(round(x1*d.width))
            y1 = int(round(y1*d.height))
            s += ' opacity %0.5f %4d %4d %4d %4d'%(c,x0,y0,x1,y1)
          prediction_string.append(s)
    df_image.loc[:, 'PredictionString'] = prediction_string
    df_image.to_csv(submit_dir + '/wbf_sub_v2.csv', index=False)



In [9]:
def run_wbf_cv(nms_objectness_threshold=0.001,nms_pre_max_num=500,iou_thr=0.55,skip_box_thr=0.001):
  log = Logger()
  log.open(OUTPUT_DIR['NORM']+EXPERIMENT + '/log.wbf_cv.txt', mode='a')
  for fold in FOLDS_SET:
    out_dir = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d'%fold
    initial_checkpoint =out_dir + '/checkpoint/'+INITIAL_CHECKPOINTS[fold]
    submit_dir = out_dir + '/valid/local'
    os.makedirs(submit_dir, exist_ok=True)
    df_annotate, df_train, df_valid = make_fold('train-%d' % fold)
    valid_dataset = SiimDataset(df_annotate, df_valid)
    valid_loader  = DataLoader(
                valid_dataset,
                sampler = SequentialSampler(valid_dataset),
                batch_size  = 4,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = null_collate,
            )
    net = Net().cuda()
    net.load_state_dict(torch.load(initial_checkpoint)['state_dict'], strict=True)
    df_image = pd.DataFrame()
    df_image.loc[:,'id'] = df_valid.image + '_image'
    img_i=0
    prediction_string=[]
    df_predict = {
                    'ImageID':[],
                    'LabelName':[],
                    'Conf':[],
                    'XMin':[],
                    'XMax':[],
                    'YMin':[],
                    'YMax':[],
                }
    for t, batch in enumerate(valid_loader):
      image  = batch['image'].cuda()
      net.eval()
      with torch.no_grad():
        predict = net(image)
        predict = infer_prediction(predict)
        predict_flat = pyramid_to_flat(predict)
        batch_size = len(batch['index'])
        for b in range(batch_size):
          d = df_valid.iloc[img_i]
          #print(d.image,batch['d'][b]['image'],'\n')
          img_i=img_i+1
          p=predict_flat[b]
          i = p[..., 4] > nms_objectness_threshold
          p=p[i]
          num=len(p)
          if num==0:
            prediction_string.append('none 1 0 0 1 1')
            continue
          boxes = xywh2xyxy(p[:, :4])
          scores = p[:,4]
          if num > nms_pre_max_num:  # excess boxes
            i = scores.argsort(descending=True)[:nms_pre_max_num]  # sort by confidence
            boxes = boxes[i]
            scores = scores[i]
          labels = torch.zeros_like(scores)

          a_boxes = [(boxes/image_size).tolist()]
          a_scores = [scores.tolist()]
          a_labels = [labels.tolist()]

          wbf_boxes, wbf_scores, wbf_labels = weighted_boxes_fusion(a_boxes,a_scores,a_labels,weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
          nms_boxes, nms_scores, nms_labels = nms(a_boxes,a_scores,a_labels, weights=None, iou_thr=iou_thr)
          soft_nms_boxes, soft_nms_scores, soft_nms_labels = soft_nms(a_boxes,a_scores,a_labels, weights=None,iou_thr=iou_thr, sigma=0.1, thresh=skip_box_thr)
          nmw_boxes, nmw_scores, nmw_labels = non_maximum_weighted(a_boxes,a_scores,a_labels,weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)

          final_boxes, final_scores, final_labels = weighted_boxes_fusion([wbf_boxes,nms_boxes,soft_nms_boxes,nmw_boxes],\
                                                  [wbf_scores, nms_scores,soft_nms_scores,nmw_scores],\
                                                  [wbf_labels,nms_labels,soft_nms_labels,nmw_labels],\
                                                  weights=[1,1,1,1], iou_thr=iou_thr, skip_box_thr=skip_box_thr)

          det = np.concatenate([final_boxes,final_scores[:,None]],-1)
          s=''
          for x0, y0, x1, y1, c in det:
            x0 = int(round(x0*d.width) )
            y0 = int(round(y0*d.height))
            x1 = int(round(x1*d.width))
            y1 = int(round(y1*d.height))
            s += ' opacity %0.5f %4d %4d %4d %4d'%(c,x0,y0,x1,y1)
            df_predict['ImageID'].append(d.image)
            df_predict['LabelName'].append(0)
            df_predict['Conf'].append(c)
            df_predict['XMin'].append(x0)
            df_predict['XMax'].append(x1)
            df_predict['YMin'].append(y0)
            df_predict['YMax'].append(y1)
          prediction_string.append(s)
    df_image.loc[:, 'PredictionString'] = prediction_string
    df_image.to_csv(submit_dir + '/wbf_cv_v2.csv', index=False)
    df_predict = pd.DataFrame(df_predict)
    print('df_predict.shape : %s\n' % str(df_predict.shape))

    df_truth = {
                    'ImageID':[],
                    'LabelName':[],
                    'XMin':[],
                    'XMax':[],
                    'YMin':[],
                    'YMax':[],
                }

    gb = df_annotate.groupby('image_id')
    for i,d in df_valid.iterrows():
      g = gb.get_group(d.image)
      for j,f in g.iterrows():
        if f.class_id==0: continue
        x0 = f.x
        y0 = f.y
        x1 = f.x+f.w
        y1 = f.y+f.h
        df_truth['ImageID'].append(d.image)
        df_truth['LabelName'].append(0)
        df_truth['XMin'].append(x0)
        df_truth['XMax'].append(x1)
        df_truth['YMin'].append(y0)
        df_truth['YMax'].append(y1)

    df_truth = pd.DataFrame(df_truth)
    print('df_truth.shape : %s\n' % str(df_truth.shape))

    map, _ = mean_average_precision_for_boxes(df_truth, df_predict, iou_threshold=0.5, exclude_not_in_annotations=False, verbose=False)

    log_str = f'f{fold}; map: {map}; map/6 {map/6}; pre_max: {nms_pre_max_num}; o/b_thr: {skip_box_thr}; w_thr: {iou_thr};\n'

    log.write(log_str)


In [10]:
def run_ensemble_submit():

    for fold in FOLDS_SET:
        out_dir = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d'%fold
        initial_checkpoint =out_dir + '/checkpoint/'+INITIAL_CHECKPOINTS[fold] # None #
            
        if 1:
            ## setup  ----------------------------------------
            #mode = 'local'
            mode = 'remote'

            submit_dir = out_dir + '/valid/%s'%(mode)
            os.makedirs(submit_dir, exist_ok=True)

            log = Logger()
            log.open(out_dir + '/log.submit.txt', mode='a')
            log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
            log.write('\t%s\n' % COMMON_STRING)
            log.write('\n')

            #
            ## dataset ------------------------------------

            if 'remote' in mode: #1263
                df_annotate, df_valid = make_fold('test')


            valid_dataset = SiimDataset(df_annotate, df_valid)
            valid_loader  = DataLoader(
                valid_dataset,
                sampler = SequentialSampler(valid_dataset),
                batch_size  = 4,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = null_collate,
            )
            log.write('mode : %s\n'%(mode))
            log.write('valid_dataset : \n%s\n'%(valid_dataset))

            ## net ----------------------------------------
            if 1:
                net = Net().cuda()
                net.load_state_dict(torch.load(initial_checkpoint)['state_dict'], strict=True)

                #---
                start_timer = timer()
                detection = do_predict(net, valid_loader)
                log.write('time %s \n' % time_to_str(timer() - start_timer, 'min'))
                log.write('detection %d \n' % len(detection))

                write_pickle_to_file(submit_dir + '/detection.pickle',detection)
                #df_valid['study'].to_csv(submit_dir + '/study.csv', index=False)
                #df_valid.to_csv(submit_dir + '/df_valid.csv', index=False)

                #write_pickle_to_file(submit_dir + '/study.pickle', df_valid.study.values)
                #exit(0)
            else:
                detection = read_pickle_from_file(submit_dir + '/detection.pickle')
                pass


            #----
            df_image  = make_df_image(df_valid, detection)
            df_submit = df_image
            if (fold==FOLDS_SET[0]):
              df_ens_submit = df_submit.copy()
            else:
              df_ens_submit['PredictionString']+=(' '+df_submit['PredictionString'])
            df_submit.to_csv(submit_dir + '/submit.csv', index=False)

            log.write('submit_dir : %s\n' % (submit_dir))
            log.write('initial_checkpoint : t%s\n' % (initial_checkpoint))
            log.write('df_submit : %s\n' % str(df_submit.shape))
            log.write('%s\n' % str(df_submit))
            log.write('\n')
    df_ens_submit.to_csv(OUTPUT_DIR['NORM']+EXPERIMENT+ENSEMBLE_CSV,index=False)    

In [11]:
def solve_bbox_problems(bbox_v, scores_v, labels_v):
    """ 
    Solves problems in the "ensemble-boxes" way 
    """
    
    to_remove = np.zeros(bbox_v.shape[0], dtype=np.bool)
    for i in range(bbox_v.shape[0]):
        x1, y1, x2, y2 = bbox_v[i]
        
        if x2 < x1:
#             warnings.warn('X2 < X1 value in box. Swap them.')
            x1, x2 = x2, x1
        if y2 < y1:
#             warnings.warn('Y2 < Y1 value in box. Swap them.')
            y1, y2 = y2, y1
        if x1 < 0:
#             warnings.warn('X1 < 0 in box. Set it to 0.')
            x1 = 0
        if x1 > 1:
#             warnings.warn('X1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
            x1 = 1
        if x2 < 0:
#             warnings.warn('X2 < 0 in box. Set it to 0.')
            x2 = 0
        if x2 > 1:
#             warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
            x2 = 1
        if y1 < 0:
# # #             warnings.warn('Y1 < 0 in box. Set it to 0.')
            y1 = 0
        if y1 > 1:
# #             warnings.warn('Y1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
            y1 = 1
        if y2 < 0:
#             warnings.warn('Y2 < 0 in box. Set it to 0.')
            y2 = 0
        if y2 > 1:
#             warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
            y2 = 1
        if (x2 - x1) * (y2 - y1) == 0.0:
#             warnings.warn("Zero area box skipped: {}.".format(box_part))
            to_remove[i] = True
    
        bbox_v[i] = x1, y1, x2, y2
    
    if to_remove.sum() > 0:
        # Hack to remove bboxes using min confidence th
        bbox_v[to_remove] = np.array([0.0, 0.0, 1.0, 1.0])
        scores_v[to_remove] = 0.0
        
    return bbox_v, scores_v, labels_v

In [17]:
def run_ensemble(nms_objectness_threshold=0.001,nms_pre_max_num=500,iou_thr=0.55,skip_box_thr=0.001):
  nets = []
  #sub_0598 = pd.read_csv('/content/submit_0598.csv')
  #sub_0598 = sub_0598.set_index('id')
  submit_dir =  OUTPUT_DIR['NORM']+EXPERIMENT
  df_annotate, df_valid = make_fold('test')
  valid_dataset = SiimDataset(df_annotate, df_valid)
  valid_loader  = DataLoader(
                valid_dataset,
                sampler = SequentialSampler(valid_dataset),
                batch_size  = 4,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = null_collate,
            )
  
  for fold in FOLDS_SET:
      out_dir = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d'%fold
      initial_checkpoint =out_dir + '/checkpoint/'+INITIAL_CHECKPOINTS[fold]
      net = Net().cuda()
      net.load_state_dict(torch.load(initial_checkpoint)['state_dict'], strict=True)
      nets.append(net)

  df_image = pd.DataFrame()
  df_image.loc[:,'id'] = df_valid.image + '_image'
  img_i=0
  prediction_string=[]
  for t, batch in enumerate(valid_loader):
      image  = batch['image'].cuda()
      batch_size = len(batch['index'])
      predict_flat_list=[]
      for net in nets:
          net.eval()
          with torch.no_grad():
            predict = net(image)
            predict = infer_prediction(predict)
            predict_flat = pyramid_to_flat(predict)
            predict_flat_list.append(predict_flat)

      for b in range(batch_size):
        d = df_valid.iloc[img_i]
        img_i +=1
        boxes_list=[]
        scores_list=[]
        labels_list=[]
        for j in range(len(nets)):
                p=predict_flat_list[j][b]
                i = p[..., 4] > nms_objectness_threshold
                p=p[i]
                num=len(p)
                if num==0:
                  prediction_string.append('none 1 0 0 1 1')
                  continue
                boxes = xywh2xyxy(p[:, :4])
                scores = p[:,4]
                if num > nms_pre_max_num:
                    i = scores.argsort(descending=True)[:nms_pre_max_num]  # sort by confidence
                    boxes = boxes[i]
                    scores = scores[i]

                labels = torch.zeros_like(scores)

                boxes_list.append((boxes/image_size).tolist())
                scores_list.append(scores.tolist())
                labels_list.append(labels.tolist())

        #line_0598=sub_0598.loc[[d.image+'_image']].PredictionString.values[0]
        #bset_0598=re.findall('opacity [+-]?\d+\.?\d* [+-]?\d+ [+-]?\d+ [+-]?\d+ [+-]?\d+',line_0598)
        #boxes_0598=[]
        #scores_0598=[]
        #for sample in bset_0598:
        #  spl = sample.split(' ')
        #  boxes_0598.append([(int(spl[2])/d.width),
        #                     (int(spl[3])/d.height),
        #                     (int(spl[4])/d.width),
        #                     (int(spl[5])/d.height)])
        #  scores_0598.append(float(spl[1]))
        
        #labels_0598 = [0 for k in range(len(scores_0598))]
        #boxes_list,scores_list,labels_list = solve_bbox_problems( boxes_list,scores_list,labels_list)
        w = [1.0 for k in range(len(nets))]
        
        #nms_boxes, nms_scores, nms_labels = nms(boxes_list,scores_list,labels_list, weights=w, iou_thr=0.5)
        #wbf_boxes, wbf_scores, wbf_labels = weighted_boxes_fusion(boxes_list,scores_list,labels_list,
        #                                                         weights=w, iou_thr=0.575, skip_box_thr=skip_box_thr)
        #soft_nms_boxes, soft_nms_scores, soft_nms_labels = soft_nms(boxes_list,scores_list,labels_list, weights=w,
        #                                                            iou_thr=0.5, sigma=0.1, thresh=skip_box_thr)
        nmw_boxes, nmw_scores, nmw_labels = non_maximum_weighted(boxes_list,scores_list,labels_list,weights=w, 
                                                                 iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        #final_boxes, final_scores, final_labels = non_maximum_weighted([nmw_boxes,boxes_0598],
        #                                                               [nmw_scores,scores_0598],
        #                                                               [nmw_labels,labels_0598],
        #                                                               weights=[1,1], iou_thr=0.5, skip_box_thr=skip_box_thr)
        
        #final_boxes, final_scores, final_labels = weighted_boxes_fusion([wbf_boxes,nms_boxes,soft_nms_boxes,nmw_boxes],\
        #                                          [wbf_scores, nms_scores,soft_nms_scores,nmw_scores],\
        #                                          [wbf_labels,nms_labels,soft_nms_labels,nmw_labels],\
        #                                         weights=[1,1,1,1], iou_thr=0.575, skip_box_thr=skip_box_thr)
        
        


            
        det = np.concatenate([nmw_boxes,nmw_scores[:,None]],-1)
        s=''
        for x0, y0, x1, y1, c in det:
            x0 = int(round(x0*d.width) )
            y0 = int(round(y0*d.height))
            x1 = int(round(x1*d.width))
            y1 = int(round(y1*d.height))
            s += ' opacity %0.5f %4d %4d %4d %4d'%(c,x0,y0,x1,y1)
        prediction_string.append(s)

  df_image.loc[:, 'PredictionString'] = prediction_string
  df_image.to_csv(submit_dir + '/nmw_5f500_iou55.csv', index=False)


In [13]:
#run_submit()

In [14]:
#run_wbf_cv()

In [18]:
run_ensemble(nms_pre_max_num=500)

/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_nmw.py:70: UserWarning: X1 < 0 in box. Set it to 0.
  warnings.warn('X1 < 0 in box. Set it to 0.')
/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_nmw.py:82: UserWarning: Y1 < 0 in box. Set it to 0.
  warnings.warn('Y1 < 0 in box. Set it to 0.')
/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_nmw.py:79: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_nmw.py:91: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
